In [5]:
import pandas as pd

df = pd.read_csv('./user-tweets.csv')

df.groupby(['username']).size().sort_values(ascending=False)

username
medical_xpress    100
slashdot          100
Rainmaker1973     100
PopMech           100
archillect        100
                 ... 
paraga              1
jack                1
PyTorch             1
wlopwangling        1
bbccomedy           1
Length: 88, dtype: int64